In [2]:
pip install langchain chromadb pandas openai


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.9.2 which is incompatible.
spacy 3.4.4 requires typer<0.8.0,>=0.3.0, but you have typer 0.12.5 which is incompatible.



  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached typer-0.12.5-py3-none-any.whl.metadata (15 kB)
  Using cached graphlib_backport-1.1.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached tomli-2.0.2-py3-none-any.whl.metadata (10.0 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached importlib_metadata-8.4.0-py3-none-any.whl.metadata (4.7 kB)

In [1]:
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import chromadb

# Initialize the Sentence Transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load CVE dataset
def load_data(filepath):
    df = pd.read_csv(filepath)
    print("Column Names:", df.columns.tolist())
    return df

# Function to create LangChain Documents
def create_documents(df):
    documents = []
    for _, row in df.iterrows():
        # Concatenate all features except 'CVE ID' into a single document content
        content = (
            f"Source Identifier: {row['Source Identifier']}\n"
            f"Published Date: {row['Published Date']}\n"
            f"Last Modified Date: {row['Last Modified Date']}\n"
            f"Vulnerability Status: {row['Vulnerability Status']}\n"
            f"Description: {row['Description']}\n"
            f"CVSS Score: {row['CVSS Score']}\n"
            f"Weaknesses: {row['Weaknesses']}\n"
            f"Configuration: {row['Configuration']}\n"
            f"References: {row['References']}\n"
            f"Entities: {row['Entities']}\n"
            f"Dependencies: {row['Dependencies']}\n"
            f"Category: {row['Category']}"
        )
        documents.append(Document(page_content=content, metadata={"CVE ID": row['CVE ID']}))
    return documents

# Split documents into smaller chunks
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Initialize ChromaDB and store embeddings
def store_embeddings_chroma(text_chunks):
    # Initialize Chroma client
    client = chromadb.Client()
    # Create a collection to store CVE documents
    collection = client.get_or_create_collection("cve_data_with_categories.csv")

    # Generate embeddings and store in ChromaDB
    for chunk in text_chunks:
        embedding = model.encode(chunk.page_content).tolist()
        collection.add(
            documents=[{
                "id": chunk.metadata["CVE ID"],
                "embedding": embedding,
                "metadata": chunk.metadata,
                "content": chunk.page_content  # Store content for retrieval
            }]
        )
    return collection

# Retrieve CVE details by CVE ID
def retrieve_cve_data(collection, cve_id_query):
    # Query by the CVE ID using the metadata in ChromaDB
    results = collection.get(metadata={"CVE ID": cve_id_query})
    return results

def main():
    # Load data
    filepath = "cve_data_with_categories.csv"  # Update this with the path to your CSV file
    df = load_data(filepath)

    # Prepare documents
    documents = create_documents(df)

    # Split into chunks
    text_chunks = text_split(documents)
    print("Length of Text Chunks:", len(text_chunks))

    # Store embeddings in Chromadb
    collection = store_embeddings_chroma(text_chunks)

    # Input CVE ID for retrieval
    cve_id_query = "CVE-2022-12345"  # Replace with the desired CVE ID
    results = retrieve_cve_data(collection, cve_id_query)

    # Display the results
    if results:
        print(f"Details for CVE ID {cve_id_query}:")
        for result in results:
            print(result["content"])  # Print out the result content
    else:
        print(f"No details found for CVE ID {cve_id_query}")

if __name__ == "__main__":
    main()


Column Names: ['CVE ID', 'Source Identifier', 'Published Date', 'Last Modified Date', 'Vulnerability Status', 'Description', 'CVSS Score', 'Weaknesses', 'Configuration', 'References', 'Entities', 'Dependencies', 'Category']
Length of Text Chunks: 1286473


TypeError: add() missing 1 required positional argument: 'ids'

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import chromadb
import os

# Initialize the Sentence Transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load CVE dataset
def load_data(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"The file {filepath} does not exist.")
    df = pd.read_csv(filepath)
    print("Column Names:", df.columns.tolist())
    return df

# Function to create LangChain Documents
def create_documents(df):
    documents = []
    for _, row in df.iterrows():
        content = (
            f"Source Identifier: {row['Source Identifier']}\n"
            f"Published Date: {row['Published Date']}\n"
            f"Last Modified Date: {row['Last Modified Date']}\n"
            f"Vulnerability Status: {row['Vulnerability Status']}\n"
            f"Description: {row['Description']}\n"
            f"CVSS Score: {row['CVSS Score']}\n"
            f"Weaknesses: {row['Weaknesses']}\n"
            f"Configuration: {row['Configuration']}\n"
            f"References: {row['References']}\n"
            f"Entities: {row['Entities']}\n"
            f"Dependencies: {row['Dependencies']}\n"
            f"Category: {row['Category']}"
        )
        documents.append(Document(page_content=content, metadata={"CVE ID": row['CVE ID']}))
    return documents

# Split documents into smaller chunks
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Initialize ChromaDB and store embeddings
def store_embeddings_chroma(text_chunks):
    client = chromadb.Client()
    collection = client.get_or_create_collection("cve_data")

    # Generate embeddings and store in ChromaDB
    ids = []  # List to store IDs
    embeddings = []  # List to store embeddings
    metadatas = []  # List to store metadata
    contents = []  # List to store content

    for i, chunk in enumerate(text_chunks):
        embedding = model.encode(chunk.page_content).tolist()
        ids.append(f"{chunk.metadata['CVE ID']}_{i}")  # Unique ID for each chunk
        embeddings.append(embedding)
        metadatas.append(chunk.metadata)
        contents.append(chunk.page_content)

    # Add all at once
    collection.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=metadatas,
        documents=contents
    )
    return collection

# Retrieve CVE details by CVE ID
def retrieve_cve_data(collection, cve_id_query):
    results = collection.similarity_search(cve_id_query, k=1)  # Perform similarity search
    return results

def main():
    filepath = r"E:\NLP\CVE_Database-main\cve_data_with_categories.csv"  # Updated path to your CSV file
    try:
        df = load_data(filepath)

        # Prepare documents
        documents = create_documents(df)

        # Split into chunks
        text_chunks = text_split(documents)
        print("Length of Text Chunks:", len(text_chunks))

        # Store embeddings in ChromaDB
        collection = store_embeddings_chroma(text_chunks)

        # Input CVE ID for retrieval
        cve_id_query = "CVE-2022-12345"  # Replace with the desired CVE ID
        results = retrieve_cve_data(collection, cve_id_query)

        # Display the results
        if results:
            print(f"Details for CVE ID {cve_id_query}:")
            for result in results:
                print(result["content"])  # Print out the result content
        else:
            print(f"No details found for CVE ID {cve_id_query}")
    except FileNotFoundError as e:
        print(e)

if __name__ == "__main__":
    main()


e:\maniconda\envs\fai\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Column Names: ['CVE ID', 'Source Identifier', 'Published Date', 'Last Modified Date', 'Vulnerability Status', 'Description', 'CVSS Score', 'Weaknesses', 'Configuration', 'References', 'Entities', 'Dependencies', 'Category']


KeyboardInterrupt: 

In [2]:
# Specify a persistent directory for ChromaDB data storage
client = chromadb.Client(path="path/to/chroma_data")


TypeError: Client() got an unexpected keyword argument 'path'

In [4]:
from langchain.vectorstores import Chroma

# Set up the directory for persistent storage
PERSIST_DIRECTORY = "path/to/chroma_data"

# Initialize Chroma vector store with persistent storage
vector_store = Chroma.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    persist_directory=PERSIST_DIRECTORY
)


NameError: name 'text_chunks' is not defined

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import chromadb
import os

# Initialize the Sentence Transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load CVE dataset
def load_data(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"The file {filepath} does not exist.")
    df = pd.read_csv(filepath)
    print("Column Names:", df.columns.tolist())
    return df

# Function to create LangChain Documents
def create_documents(df):
    documents = []
    for _, row in df.iterrows():
        content = (
            f"Source Identifier: {row['Source Identifier']}\n"
            f"Published Date: {row['Published Date']}\n"
            f"Last Modified Date: {row['Last Modified Date']}\n"
            f"Vulnerability Status: {row['Vulnerability Status']}\n"
            f"Description: {row['Description']}\n"
            f"CVSS Score: {row['CVSS Score']}\n"
            f"Weaknesses: {row['Weaknesses']}\n"
            f"Configuration: {row['Configuration']}\n"
            f"References: {row['References']}\n"
            f"Dependencies: {row['Dependencies']}\n"
            f"Category: {row['Category']}"
        )
        documents.append(Document(page_content=content, metadata={"CVE ID": row['CVE ID']}))
    return documents

# Split documents into smaller chunks
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Initialize ChromaDB and store embeddings with batch processing
def store_embeddings_chroma(text_chunks, persist_directory="chroma_data"):
    # Initialize Chroma vector store with persistent storage
    vector_store = Chroma(persist_directory=persist_directory)

    # Prepare batches for embedding generation
    batch_size = 32  # Adjust batch size based on available memory and model capacity
    ids = []
    embeddings = []
    metadatas = []
    contents = []

    # Process embeddings in batches
    for i in range(0, len(text_chunks), batch_size):
        batch_chunks = text_chunks[i:i + batch_size]
        batch_contents = [chunk.page_content for chunk in batch_chunks]
        batch_embeddings = model.encode(batch_contents)  # Generate embeddings in a single batch

        for j, chunk in enumerate(batch_chunks):
            ids.append(f"{chunk.metadata['CVE ID']}_{i + j}")
            embeddings.append(batch_embeddings[j].tolist())
            metadatas.append(chunk.metadata)
            contents.append(chunk.page_content)

    # Store all embeddings at once
    vector_store.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=metadatas,
        documents=contents
    )
    vector_store.persist()
    return vector_store

# Retrieve CVE details by CVE ID
def retrieve_cve_data(vector_store, cve_id_query):
    results = vector_store.similarity_search(cve_id_query, k=1)
    return results

def main():
    filepath = r"E:\NLP\CVE_Database-main\cve_data_with_categories.csv"  # Path to your CSV file
    try:
        df = load_data(filepath)

        # Prepare documents
        documents = create_documents(df)

        # Split into chunks
        text_chunks = text_split(documents)
        print("Length of Text Chunks:", len(text_chunks))

        # Store embeddings in ChromaDB with batch processing
        vector_store = store_embeddings_chroma(text_chunks)

        # Input CVE ID for retrieval
        cve_id_query = "CVE-2022-12345"  # Replace with the desired CVE ID
        results = retrieve_cve_data(vector_store, cve_id_query)

        # Display the results
        if results:
            print(f"Details for CVE ID {cve_id_query}:")
            for result in results:
                print(result["content"])  # Print out the result content
        else:
            print(f"No details found for CVE ID {cve_id_query}")
    except FileNotFoundError as e:
        print(e)

if __name__ == "__main__":
    main()


Column Names: ['CVE ID', 'Source Identifier', 'Published Date', 'Last Modified Date', 'Vulnerability Status', 'Description', 'CVSS Score', 'Weaknesses', 'Configuration', 'References', 'Entities', 'Dependencies', 'Category']
Length of Text Chunks: 1247529


C:\Users\imaba\AppData\Local\Temp\ipykernel_14792\2560265177.py:49: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma(persist_directory=persist_directory)


KeyboardInterrupt: 

In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os

# Initialize the Sentence Transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load CVE dataset
def load_data(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"The file {filepath} does not exist.")
    df = pd.read_csv(filepath)
    print("Loaded data with columns:", df.columns.tolist())
    return df

# Function to create LangChain Documents
def create_documents(df):
    documents = []
    for _, row in df.iterrows():
        content = (
            f"Source Identifier: {row['Source Identifier']}\n"
            f"Published Date: {row['Published Date']}\n"
            f"Last Modified Date: {row['Last Modified Date']}\n"
            f"Vulnerability Status: {row['Vulnerability Status']}\n"
            f"Description: {row['Description']}\n"
            f"CVSS Score: {row['CVSS Score']}\n"
            f"Weaknesses: {row['Weaknesses']}\n"
            f"Configuration: {row['Configuration']}\n"
            f"References: {row['References']}\n"
            f"Dependencies: {row['Dependencies']}\n"
            f"Category: {row['Category']}"
        )
        documents.append(Document(page_content=content, metadata={"CVE ID": row['CVE ID']}))
    return documents

# Split documents into smaller chunks
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Initialize ChromaDB and store embeddings with batch processing
def store_embeddings_chroma(text_chunks, persist_directory="chroma_data", batch_size=500):
    # Initialize Chroma vector store with persistent storage
    vector_store = Chroma(persist_directory=persist_directory)

    # Process embeddings in batches to manage memory
    for i in range(0, len(text_chunks), batch_size):
        batch_chunks = text_chunks[i:i + batch_size]
        batch_contents = [chunk.page_content for chunk in batch_chunks]
        batch_embeddings = model.encode(batch_contents)  # Generate embeddings in a batch

        ids = []
        embeddings = []
        metadatas = []
        contents = []

        for j, chunk in enumerate(batch_chunks):
            ids.append(f"{chunk.metadata['CVE ID']}_{i + j}")
            embeddings.append(batch_embeddings[j].tolist())
            metadatas.append(chunk.metadata)
            contents.append(chunk.page_content)

        # Add batch to vector store and persist
        vector_store.add(
            ids=ids,
            embeddings=embeddings,
            metadatas=metadatas,
            documents=contents
        )
        vector_store.persist()  # Persist batch to disk

        print(f"Processed batch {i // batch_size + 1}/{(len(text_chunks) // batch_size) + 1}")

    return vector_store

# Retrieve CVE details by CVE ID
def retrieve_cve_data(vector_store, cve_id_query):
    results = vector_store.similarity_search(cve_id_query, k=1)
    return results

def main():
    filepath = r"E:\NLP\CVE_Database-main\cve_data_with_categories.csv"  # Path to your CSV file
    try:
        df = load_data(filepath)

        # Prepare documents
        documents = create_documents(df)

        # Split into chunks
        text_chunks = text_split(documents)
        print("Total Text Chunks:", len(text_chunks))

        # Store embeddings in ChromaDB with batch processing
        vector_store = store_embeddings_chroma(text_chunks)

        # Input CVE ID for retrieval
        cve_id_query = "CVE-2022-12345"  # Replace with the desired CVE ID
        results = retrieve_cve_data(vector_store, cve_id_query)

        # Display the results
        if results:
            print(f"Details for CVE ID {cve_id_query}:")
            for result in results:
                print(result["content"])  # Print out the result content
        else:
            print(f"No details found for CVE ID {cve_id_query}")
    except FileNotFoundError as e:
        print(e)

if __name__ == "__main__":
    main()


Loaded data with columns: ['CVE ID', 'Source Identifier', 'Published Date', 'Last Modified Date', 'Vulnerability Status', 'Description', 'CVSS Score', 'Weaknesses', 'Configuration', 'References', 'Entities', 'Dependencies', 'Category']
Total Text Chunks: 1247529


AttributeError: 'Chroma' object has no attribute 'add'

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import chromadb
import os

# Initialize the Sentence Transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load CVE dataset
def load_data(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"The file {filepath} does not exist.")
    df = pd.read_csv(filepath)
    print("Column Names:", df.columns.tolist())
    return df

# Function to create LangChain Documents
def create_documents(df):
    documents = []
    for _, row in df.iterrows():
        content = (
            f"Source Identifier: {row['Source Identifier']}\n"
            f"Published Date: {row['Published Date']}\n"
            f"Last Modified Date: {row['Last Modified Date']}\n"
            f"Vulnerability Status: {row['Vulnerability Status']}\n"
            f"Description: {row['Description']}\n"
            f"CVSS Score: {row['CVSS Score']}\n"
            f"Weaknesses: {row['Weaknesses']}\n"
            f"Configuration: {row['Configuration']}\n"
            f"References: {row['References']}\n"
            f"Dependencies: {row['Dependencies']}\n"
            f"Category: {row['Category']}"
        )
        documents.append(Document(page_content=content, metadata={"CVE ID": row['CVE ID']}))
    return documents

# Split documents into manageable chunks
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Initialize ChromaDB and store embeddings with batching
def store_embeddings_chroma(text_chunks, batch_size=64):
    client = chromadb.Client()
    collection = client.get_or_create_collection("cve_data")

    # Process and add embeddings in batches
    for i in range(0, len(text_chunks), batch_size):
        batch_chunks = text_chunks[i:i + batch_size]
        ids = [f"{chunk.metadata['CVE ID']}_{j}" for j, chunk in enumerate(batch_chunks)]
        contents = [chunk.page_content for chunk in batch_chunks]
        metadatas = [chunk.metadata for chunk in batch_chunks]
        embeddings = model.encode(contents).tolist()

        # Add batch to ChromaDB
        collection.add(
            ids=ids,
            embeddings=embeddings,
            metadatas=metadatas,
            documents=contents
        )
        print(f"Processed batch {i // batch_size + 1} / {len(text_chunks) // batch_size + 1}")
    return collection

# Retrieve CVE details by CVE ID
def retrieve_cve_data(collection, cve_id_query):
    results = collection.similarity_search(cve_id_query, k=1)  # Perform similarity search
    return results

def main():
    filepath = r"E:\NLP\CVE_Database-main\cve_data_with_categories.csv"  # Updated path to your CSV file
    try:
        df = load_data(filepath)

        # Prepare documents
        documents = create_documents(df)
        

        # Split into chunks
        text_chunks = text_split(documents)
        print("Total Text Chunks:", len(text_chunks))

        # Store embeddings in ChromaDB with batching
        collection = store_embeddings_chroma(text_chunks)

        # Input CVE ID for retrieval
        cve_id_query = "CVE-2022-12345"  # Replace with the desired CVE ID
        results = retrieve_cve_data(collection, cve_id_query)

        # Display the results
        if results:
            print(f"Details for CVE ID {cve_id_query}:")
            for result in results:
                print(result["content"])  # Print out the result content
        else:
            print(f"No details found for CVE ID {cve_id_query}")
    except FileNotFoundError as e:
        print(e)

if __name__ == "__main__":
    main()


e:\maniconda\envs\fai\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Column Names: ['CVE ID', 'Source Identifier', 'Published Date', 'Last Modified Date', 'Vulnerability Status', 'Description', 'CVSS Score', 'Weaknesses', 'Configuration', 'References', 'Entities', 'Dependencies', 'Category']
Total Text Chunks: 618232


: 